### CDO Structure
* The CDO does not consist of any tranches
* Valuation performed using Monte Carlo Simulation (10,000 simulations)
* Each simulation is CDO using 10 bonds described below

In [4]:
import numpy as np
from scipy.stats import norm, uniform

In [3]:
num_simulations = 10000
num_periods = 60
face_value = 100000
coupon_rate = 0.07
recovery_rate = 0.4
correlation = 0.25
copula_params = 2.0  # Clayton copula parameter
risk_free_rate = 0.035  # 3.5%
num_bonds = 10

In [36]:
def simulate_default_probabilities(num_bonds, num_simulations, num_periods, correlation, copula_params):
    # Generate independent standard normal random variables
    independent_normals = np.random.normal(size=(num_simulations, num_bonds, num_periods))

    # Generate correlated random variables using the Clayton copula
    correlated_normals = copula_transform(independent_normals, correlation, copula_params)

    # Transform correlated normals to uniform marginals
    uniform_marginals = norm.cdf(correlated_normals)

    # Simulate default probabilities based on the uniform marginals
    default_probabilities = 1 - uniform_marginals

    return default_probabilities

def copula_transform(normals, rho, copula_params):
    """
    Clayton copula transformation function.
    """
    num_columns = normals.shape[2]
    print(num_columns)
    transformed_variables = np.empty_like(normals)

    for i in range(num_columns-1):
        u = normals[:,:, i]
        v = normals[:,:, i + 1] if i + 1 < num_columns else normals[:,:, 0]  # Wrap around to the first column for the last variable

        # Clayton copula transformation formula
        w = (u**(-rho) + v**(-rho) - 1)**(-1/rho)

        transformed_variables[:,:, i] = u
        transformed_variables[:,:, i + 1] = w if i + 1 < num_columns else w  # Wrap around

    return transformed_variables



In [37]:

default_probabilities = simulate_default_probabilities(num_bonds, num_simulations, num_periods, correlation, copula_params)

# Print the first 5 simulated default probabilities for each period
print("Simulated Default Probabilities:")
print(default_probabilities[:1, :,:])
print(default_probabilities.shape)

60


/var/folders/xc/6c2pzz2172x4x83lp41y7csm0000gn/T/ipykernel_36993/2381541489.py:29: RuntimeWarning: invalid value encountered in power
  w = (u**(-rho) + v**(-rho) - 1)**(-1/rho)


Simulated Default Probabilities:
[[[0.35607763 0.4038913  0.29029463 0.68457906 0.16068226 0.31577626
   0.7236377  0.58168278 0.59158119 0.77275211 0.33792361 0.31866421
   0.78042449 0.84353546 0.4839291  0.90959403 0.58351052 0.66417881
   0.81208674 0.97630874 0.96428456 0.47641897 0.66475653 0.1504678
   0.36904368 0.89499329 0.96048631 0.25705482 0.90034341 0.85791056
   0.39852577 0.74490478 0.69141865 0.77471739 0.30542721 0.25506349
   0.33020158 0.95267501 0.32396922 0.49629242 0.75351504 0.25858423
   0.06983101 0.94066788 0.87417346 0.93119503 0.26685358 0.47839509
   0.86467183 0.75096541 0.50400998 0.71255324 0.47092157 0.00661527
   0.30735547 0.43799161 0.71858433 0.27936003 0.51200249        nan]
  [0.7599189  0.31383231 0.58467862 0.49835041 0.80086408 0.70808661
   0.46662562 0.10877648 0.04835168 0.29681466 0.72668112 0.80276757
   0.90538671 0.37851872 0.06989998 0.20302834 0.76706283 0.9426638
   0.30105788 0.28189675 0.36906822 0.82748758 0.36812173 0.11375295
  

In [38]:
def generate_default_scenarios(default_probabilities):
    """
    Generate default scenarios based on simulated default probabilities.
    """
    num_simulations, num_bonds, num_periods = default_probabilities.shape

    # Compare default probabilities to determine defaults
    default_scenarios = 0.9 < default_probabilities

    return default_scenarios

In [51]:
default_scenarios = generate_default_scenarios(default_probabilities)

In [52]:
# Print the first 5 default scenarios for each period
print("Default Scenarios:")
print(default_scenarios[:1,:, :].astype(int))
print(default_scenarios.shape)

Default Scenarios:
[[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0
   0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0
   0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0]
  [1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0
   1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0]
  [1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0
   0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
   1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0
   0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
   0 0 0 0 0 0 0 0 

In [42]:
def first_one(A):
    if len(A) == 0:
        return -1

    return first_one_subarray(A, 0, len(A) - 1)

def first_one_subarray(A, start, end):
    # Incorrect subarray
    if start > end or start > len(A) - 1:
        return -1

    # Check if 1 is on 'first' position
    if A[start] == 1:
        return start

    # Divide into two parts
    split_point = max(start + 1, round(end / 2))
    result_left = first_one_subarray(A, start + 1, split_point)
    result_right = first_one_subarray(A, split_point + 1, end)

    if result_left != -1:
        return result_left
    else:
        return result_right

In [45]:
default_times = np.zeros((num_simulations,num_bonds,num_periods ))
for i in range(num_simulations):
    for j in range(num_bonds):
        default_times[i,j,:]=first_one(default_scenarios[i,j,:])

In [102]:
# Initialize CDO values
cdo_values = np.zeros(num_simulations)

In [103]:
cdo_values.shape

(10000,)

In [58]:
default_scenarios.shape

(10000, 10, 60)

In [61]:
def calculate_cdo_cash_flows(default_scenarios, face_value, coupon_rate, recovery_rate):
    """
    Calculate CDO cash flows based on default scenarios.
    """
    num_simulations,num_bonds, num_periods = default_scenarios.shape

    # Initialize CDO cash flows matrix
    cdo_cash_flows = np.zeros((num_simulations,num_bonds, num_periods))
    #print(cdo_cash_flows[1,:])
    i=0
    for t in range(num_simulations):
        # For each Bond
        for b in range(num_bonds):
            # Calculate cash flows for each simulation at time t
            flag=True
            i=0
            for i in range(num_periods):
                #print(flag)
                if flag:
                    if default_scenarios[t,b, i] > 0.9:  # Bond defaulted
                        cdo_cash_flows[t,b, i] = face_value * (1 - recovery_rate)  # Loss given default
                        flag = False
                    else:  # Bond did not default
                        cdo_cash_flows[t,b, i] = face_value * coupon_rate / 2  # 6-month coupon payment

    return cdo_cash_flows

In [80]:
cdo_cash_flows = calculate_cdo_cash_flows(default_scenarios, face_value, coupon_rate, recovery_rate)

# Print the first 5 rows of the calculated CDO cash flows
print("CDO Cash Flows:")
print(cdo_cash_flows[:1, :])

CDO Cash Flows:
[[[ 3500.  3500.  3500.  3500.  3500.  3500.  3500.  3500.  3500.  3500.
    3500.  3500.  3500.  3500.  3500. 60000.     0.     0.     0.     0.
       0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
       0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
       0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
       0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
  [ 3500.  3500.  3500.  3500.  3500.  3500.  3500.  3500.  3500.  3500.
    3500.  3500. 60000.     0.     0.     0.     0.     0.     0.     0.
       0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
       0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
       0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
       0.     0.     0.     0.     0.     0.     0.     0.     0.     0.]
  [60000.     0.     0.     0.     0.     0.     0.     0.     0.     0.
       0.     0.     0.     0.   

In [86]:
def cash_flows_per_period_func(cdo_cash_flows):
    
    num_simulations,num_bonds, num_periods = cdo_cash_flows.shape
    cash_flows_per_period=np.zeros((num_simulations,1, num_periods))
    cash_flows_per_period = np.sum(cdo_cash_flows,axis=1)
    return cash_flows_per_period

In [104]:
cash_flows_per_period = cash_flows_per_period_func(cdo_cash_flows)
cash_flows_per_period.shape

(10000, 60)

In [105]:
cash_flows_per_period[1]

array([91500., 88000., 28000., 84500., 24500., 81000., 21000., 21000.,
       21000., 77500., 74000., 14000., 14000., 14000., 70500., 10500.,
       10500., 10500., 67000.,  7000.,  7000., 63500.,  3500.,  3500.,
        3500.,  3500.,  3500.,  3500.,  3500., 60000.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.])

In [94]:
def discount_cash_flows(cash_flows_per_period, risk_free_rate):
    """
    Discount CDO cash flows to present value using the risk-free rate.
    """
    num_simulations, num_periods = cash_flows_per_period.shape

    # Initialize discounted cash flows matrix
    discounted_cash_flows = np.zeros((num_simulations, num_periods))
    for t in range(num_periods):
        discount_factor = 1 / (1 + risk_free_rate / 2) ** (t + 1)  # 6-month compounding

        # Discount cash flows for each simulation at time t
        discounted_cash_flows[:, t] = cash_flows_per_period[:, t] * discount_factor

    return discounted_cash_flows

In [95]:
discounted_cash_flows = discount_cash_flows(cash_flows_per_period, risk_free_rate)

In [98]:
# Print the first 5 rows of the discounted cash flows
print("Discounted Cash Flows:")
print(discounted_cash_flows[1, :])

Discounted Cash Flows:
[89926.28992629 84999.00391792 26579.98782334 78834.99371963
 22464.35713628 72992.5458773  18598.51928805 18278.64303494
 17964.26833901 65156.46642206 61143.89810867 11368.81032634
 11173.277962   10981.10856216 54346.65873166  7954.97121902
  7818.15353221  7683.68897514 48185.99848691  4947.77204017
  4862.67522375 43352.73804258  2348.42358179  2308.03300422
  2268.33710488  2229.323936    2190.98175528  2153.29902239
  2116.26439547 35654.85818977     0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.        ]


In [99]:
discounted_cash_flows.shape

(10000, 60)

In [106]:
for i in range(num_simulations):
    cdo_values[i] = np.sum(discounted_cash_flows[i,:])

In [107]:
cdo_values.shape

(10000,)

In [109]:
cdo_values[:5]

array([767213.41197295, 820884.34768525, 830665.34256526, 767470.48748663,
       773862.21128138])

In [110]:
expected_cdo_value = np.mean(cdo_values)

In [111]:
expected_cdo_value

778324.9956154657

In [112]:
cdo_values

array([767213.41197295, 820884.34768525, 830665.34256526, ...,
       729117.71160652, 762653.74298893, 833658.65510083])

In [113]:
discounted_cash_flows

array([[145454.54545455,  27045.13761025,  80214.60610972, ...,
             0.        ,      0.        ,      0.        ],
       [ 89926.28992629,  84999.00391792,  26579.98782334, ...,
             0.        ,      0.        ,      0.        ],
       [ 34398.03439803,  88379.6461192 ,  29902.48630126, ...,
             0.        ,      0.        ,      0.        ],
       ...,
       [ 34398.03439803, 142952.8702256 ,  26579.98782334, ...,
             0.        ,      0.        ,      0.        ],
       [ 34398.03439803, 142952.8702256 ,  80214.60610972, ...,
             0.        ,      0.        ,      0.        ],
       [ 34398.03439803,  88379.6461192 ,  83537.10458764, ...,
             0.        ,      0.        ,      0.        ]])